# Galazy Zoo as a training set for a Galaxy Classification neural network

In [ ]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
    
# this package
from astronn.data import fetch_GalaxyZoo

%matplotlib inline

### Load up Galaxy Zoo data - contains train and validation data

In [ ]:
cache_file = fetch_GalaxyZoo()

In [ ]:
def randomize(data, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_data = data[permutation]
    shuffled_labels = labels[permutation]
    return shuffled_data, shuffled_labels

with h5py.File(cache_file, 'r') as f:
    train_dataset, train_labels = randomize(f['train']['images'][:], f['train']['labels'][:])
#    valid_dataset, valid_labels = randomize(f['validate']['images'][:], f['validate']['labels'][:])
    valid_dataset, valid_labels = randomize(f['test']['images'][0:5000], f['test']['labels'][0:5000])
    test_dataset, test_labels = randomize(f['test']['images'][5000:10000], f['test']['labels'][5000:10000])

print train_dataset.shape
print valid_dataset.shape
print test_dataset.shape
    

### Let's explore this awesome data

In [ ]:
fig,axes = plt.subplots(4,6,figsize=(8,5.3),sharex=True,sharey=True)
with h5py.File(cache_file, 'r') as f:
    for i in range(24):
        ax = axes.flat[i]
        
        idx = np.random.randint(f['test']['images'].shape[0])
        ax.imshow(f['test']['images'][idx], interpolation='nearest')
        

In [ ]:
# hdf5 data structure

with h5py.File(cache_file, 'r') as f:
    print f.filename
    print f.name
    print f.keys()
    print f['train']['images'].shape[0], "images in the training set"
    print f['test']['images'].shape[0], "images in the testing set"
    print f['validate']['images'].shape[0], "images in the validation set"
    print f['test'].name    
    print f['test'].keys()
    print f['test']['metadata'].dtype.names
    print f['test']['images'].shape
    print f['test']['labels'].shape

In [ ]:
fig,axes = plt.subplots(1,6,figsize=(12,2),sharex=True,sharey=True)

with h5py.File(cache_file, 'r') as f:
    
    n_images = f['train']['labels'].shape[0]
    
        
    gal_class = np.zeros((6, n_images))
    for i in np.arange(6):
        ax = axes.flat[i]
        ax.set_xticks([0.2, 0.5, 0.8])
        
        gal_class[i] = np.array(f['train']['labels']).T[i]
        gal_class[i] = np.sort(gal_class[i])
    
        n_50pct = len(gal_class[i][gal_class[i]>0.5])
        n_75pct = len(gal_class[i][gal_class[i]>0.75])
        n_90pct = len(gal_class[i][gal_class[i]>0.9])
            
        ax.plot(np.arange(n_images)/float(n_images), gal_class[i])
        ax.set_title(str(i) + "\nP>50%:" + str(n_50pct) + "\n" + "P>75%:" + str(n_75pct) + "\n" + "P>90%:" + str(n_90pct))
    
#plt.show()
plt.savefig("../figures/class_distribution.pdf")

### Let's create a refined training set that includes only galaxies with a classification > 50%

In [ ]:
fig,axes = plt.subplots(6,5,figsize=(8,9),sharex=True,sharey=True)

for i in np.arange(6):
        
    args = test_labels.T[i].argsort()[-5:]
    
    # Plot 5 highest probability galaxy types
    for j in np.arange(5):
        ax = axes.flat[5*i + j]
        ax.imshow(test_dataset[args[j]], interpolation='nearest')
        ax.set_xticks([0, 10, 20, 30])
        ax.set_yticks([0, 10, 20, 30])
        if j == 0: ax.set_ylabel(str(i))

#plt.savefig("../figures/class_sample.pdf")
plt.show()

In [ ]:
args = np.array([], dtype=int)

for i in np.arange(6):

    args = np.append(args, train_labels.T[i].argsort()[-1500:])
    
train_subdata, train_sublabels = randomize(train_dataset[args], train_labels[args])

print train_subdata.shape
print train_sublabels.shape

In [ ]:
fig,axes = plt.subplots(6,5,figsize=(8,9),sharex=True,sharey=True)

for i in np.arange(6):
        
    args = train_sublabels.T[i].argsort()[-5:]
        
    # Plot 5 highest probability galaxy types
    for j in np.arange(5):
        ax = axes.flat[5*i + j]
        ax.imshow(train_subdata[args[j]], interpolation='nearest')
        ax.set_xticks([0, 10, 20, 30])
        ax.set_yticks([0, 10, 20, 30])
        if j == 0: ax.set_ylabel(str(i))

plt.show()

### Set all labels to highest likelihood class

In [ ]:
def orthogonal_basis(array_in):
    array_tmp = np.zeros(array_in.shape)
    
    array_tmp[np.arange(len(array_in)),np.argmax(array_in, axis=1)] = 1

    return array_tmp
    
    
train_sublabels = orthogonal_basis(train_sublabels)
test_labels = orthogonal_basis(test_labels)
valid_labels = orthogonal_basis(valid_labels)



### Now we reformat into 1-hats

In [ ]:
image_size = 32
num_labels = 6
num_channels = 3

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size * num_channels)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
#    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_subdata, train_sublabels = reformat(train_subdata, train_sublabels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_subdata.shape, train_sublabels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

### Let's now build our neural network

In [ ]:
def accuracy_classes(predictions, labels):
    
    acc = np.zeros(num_labels)
    
    labels_test = labels[np.where(np.argmax(predictions, 1) == np.argmax(labels, 1))]
    
    predictions_test = predictions[np.where(np.argmax(predictions, 1) == np.argmax(labels, 1))]
    predictions_test = orthogonal_basis(predictions_test)
    
    for i in np.arange(num_labels):
        
        if np.sum(np.argmax(labels, 1) == i) == 0:
            acc[i] = -1.0
            print "Channel", i, "- no samples"
        else:
            acc[i] = 100.0 * np.sum(np.argmax(labels_test, 1) == i) / np.sum(np.argmax(labels, 1) == i) 
            print "Channel", i, "-", acc[i], "or", \
                            np.sum(np.argmax(labels_test, 1) == i), "of", np.sum(np.argmax(labels, 1) == i)
    
    return acc
#    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [ ]:
batch_size = 64
image_size = 32
num_labels = 6
num_channels = 3
flat_size = num_channels*image_size**2
x_patch = 5
y_patch = 5
layer1_out = 32
layer2_out = 64
layer3_fc = 1024


# Weight initialization function
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# Bias initialization function
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Convolutional function
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# Pooling function
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')




sess = tf.InteractiveSession()


x = tf.placeholder(tf.float32, shape=[None, flat_size])
y_ = tf.placeholder(tf.float32, shape=[None, num_labels])


# Reshape image into 4D tensor
x_image = tf.reshape(x, [-1,image_size,image_size,num_channels])


# Layer 1
W_conv1 = weight_variable([x_patch, y_patch, num_channels, layer1_out])
b_conv1 = bias_variable([layer1_out])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


# Layer 2
W_conv2 = weight_variable([x_patch, y_patch, layer1_out, layer2_out])
b_conv2 = bias_variable([layer2_out])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


# Fully connected layer
W_fc1 = weight_variable([8 * 8 * layer2_out, layer3_fc])
b_fc1 = bias_variable([layer3_fc])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*layer2_out])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


# Readout layer
W_fc2 = weight_variable([layer3_fc, num_labels])
b_fc2 = bias_variable([num_labels])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)



# Entropy, optimizer
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

# Accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Train the model    
sess.run(tf.initialize_all_variables())
for i in range(10000):
    
    # Get new batch
    rand_args = np.random.randint(len(train_subdata), size=batch_size)
    batch_data = train_subdata[rand_args]
    batch_labels = train_sublabels[rand_args]
    # batch = mnist.train.next_batch(50)

    if i%500 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch_data, y_: batch_labels, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch_data, y_: batch_labels, keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: test_dataset, y_: test_labels, keep_prob: 1.0}))
print("validation accuracy %g"%accuracy.eval(feed_dict={
    x: valid_dataset, y_: valid_labels, keep_prob: 1.0}))
